In [69]:
import ast
import pandas as pd

In [70]:
reviews = pd.read_parquet('DataBases/reviews.parquet')

In [71]:
reviews.head()

,user_id,user_url,reviews
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'funny': '', 'helpful': 'No ratings yet', 'i..."
1,js41637,http://steamcommunity.com/id/js41637,"[{'funny': '', 'helpful': '15 of 20 people (75..."
2,evcentric,http://steamcommunity.com/id/evcentric,"[{'funny': '', 'helpful': 'No ratings yet', 'i..."
3,doctr,http://steamcommunity.com/id/doctr,"[{'funny': '', 'helpful': '2 of 2 people (100%..."
4,maplemage,http://steamcommunity.com/id/maplemage,"[{'funny': '3 people found this review funny',..."


In [72]:
#Explotamos la columna "reviews" del dataframe en un nuevo dataframe.
df_comenExpan = reviews.explode('reviews')

#Concatenamos el dataframe original con el nuevo dataframe creado
# y elimina la columna "reviews" del dataframe concatenado
reviews = pd.concat([df_comenExpan.drop(['reviews'], axis=1), df_comenExpan['reviews'].apply(pd.Series)], axis=1)

In [73]:
names = pd.read_parquet('DataBases/id_names.parquet')

In [74]:
names['id'] = names['id'].astype(int)

In [75]:
#Renombramos para facilitar el merge.
reviews.rename(columns={'item_id': 'id'}, inplace=True)

In [76]:
#Eliminamos los nulos de la columna 'id'.
reviews = reviews.dropna(subset=['id'])

In [77]:
#Convertimos la columna 'id' en integers para facilitar el merge.
reviews['id'] = reviews['id'].astype(int)

In [78]:
#Hacemos el merge.
reviews = pd.merge(reviews, names, on='id', how='inner')
reviews.head()

,user_id,user_url,funny,helpful,id,last_edited,posted,recommend,review,0,app_name,developer
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,No ratings yet,1250,,"Posted November 5, 2011.",True,Simple yet with great replayability. In my opi...,NaN,Killing Floor,Tripwire Interactive
1,death-hunter,http://steamcommunity.com/id/death-hunter,,No ratings yet,1250,,"Posted March 30, 2015.",True,"Amazing, Non-stop action of blowing stuff to b...",NaN,Killing Floor,Tripwire Interactive
2,DJKamBer,http://steamcommunity.com/id/DJKamBer,,No ratings yet,1250,,"Posted July 12, 2013.",True,"Compared to Left 4 Dead 2, this game REALLY gi...",NaN,Killing Floor,Tripwire Interactive
3,diego9031,http://steamcommunity.com/id/diego9031,,No ratings yet,1250,,"Posted August 13, 2015.",True,Jogo ‚ô•‚ô•‚ô•‚ô•.,NaN,Killing Floor,Tripwire Interactive
4,76561198081962345,http://steamcommunity.com/profiles/76561198081...,,No ratings yet,1250,,"Posted April 5, 2014.",True,cara nas imagens esse jogo da pouco de medo ma...,NaN,Killing Floor,Tripwire Interactive


In [79]:
reviews.dtypes

user_id         object
user_url        object
funny           object
helpful         object
id               int32
last_edited     object
posted          object
recommend       object
review          object
0              float64
app_name        object
developer       object
dtype: object

In [80]:
columnas = ['id', 'app_name', 'user_id', 'funny', 'helpful', 'recommend', 'posted', 'developer', 'review']
reviews = reviews[columnas].copy()

In [81]:
reviews.head()

,id,app_name,user_id,funny,helpful,recommend,posted,developer,review
0,1250,Killing Floor,76561197970982479,,No ratings yet,True,"Posted November 5, 2011.",Tripwire Interactive,Simple yet with great replayability. In my opi...
1,1250,Killing Floor,death-hunter,,No ratings yet,True,"Posted March 30, 2015.",Tripwire Interactive,"Amazing, Non-stop action of blowing stuff to b..."
2,1250,Killing Floor,DJKamBer,,No ratings yet,True,"Posted July 12, 2013.",Tripwire Interactive,"Compared to Left 4 Dead 2, this game REALLY gi..."
3,1250,Killing Floor,diego9031,,No ratings yet,True,"Posted August 13, 2015.",Tripwire Interactive,Jogo ‚ô•‚ô•‚ô•‚ô•.
4,1250,Killing Floor,76561198081962345,,No ratings yet,True,"Posted April 5, 2014.",Tripwire Interactive,cara nas imagens esse jogo da pouco de medo ma...


In [82]:
#Ordenamos el dataframe en base a la columna 'id'
reviews = reviews.sort_values(by='id')

In [83]:
#Extraigo los anios desde la columna 'posted'.
reviews['year'] = reviews['posted'].str.extract(r'(\d{4})')

In [84]:
#Imprimo los valores unicos para poder rellenar el nulo con el anio faltante de la serie
unique_years = reviews['year'].unique()
print(unique_years)

['2014' nan '2013' '2012' '2015' '2011' '2010']


In [85]:
#Lleno los nulos con el valor faltante de la serie '2016'
# asumiendo ese valor para no eliminar todas esa data que 
# no tiene anio especificado.
reviews['year'].fillna(2016, inplace=True)

In [86]:
#Convierto la columna 'year' en integers.
reviews['year'] = reviews['year'].astype(int)

In [87]:
#Imprimo para checkear
reviews.head()


,id,app_name,user_id,funny,helpful,recommend,posted,developer,review,year
36490,10,Counter-Strike,76561198089673598,,No ratings yet,True,"Posted December 14, 2014.",Valve,o cs e um otimo jogo de fps ele quase sempre,2014
36463,10,Counter-Strike,Hype_R,,0 of 1 people (0%) found this review helpful,True,Posted January 30.,Valve,Best God Damn Game Known To Man Kind,2016
36464,10,Counter-Strike,KILLERamateur,,No ratings yet,True,"Posted March 9, 2014.",Valve,I heard that when i was made it was a huge suc...,2014
36495,10,Counter-Strike,nottynotty,,No ratings yet,True,Posted April 1.,Valve,GoODGAmP!!!,2016
36466,10,Counter-Strike,76561198082737449,,No ratings yet,True,"Posted October 18, 2014.",Valve,realmente,2014


In [88]:
import pandas as pd
from textblob import TextBlob
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer

# Descargamos los recursos de NLTK (solo es necesario hacerlo una vez)
nltk.download('vader_lexicon')

# Creamos la instancia del SentimentIntensityAnalyzer
sia = SentimentIntensityAnalyzer()

# Creamos una función para realizar el análisis de sentimiento
def analyze_sentiment(text):

    analysis = TextBlob(text)
    sentiment = analysis.sentiment.polarity

    vader_scores = sia.polarity_scores(text)
    compound_sentiment = vader_scores['compound']

    if compound_sentiment >= 0.05:
        return 2  # Positivo
    elif compound_sentiment <= -0.05:
        return 0  # Negativo
    else:
        return 1  # Neutral


# Aplicamos la función de análisis de sentimiento a la columna "review"
reviews['sentiment_analysis'] = reviews['review'].apply(analyze_sentiment)

# Mapeamos los valores numéricos a etiquetas de sentimiento
sentiment_mapping = {
    0: 'Negativo',
    1: 'Neutral',
    2: 'Positivo'
}
reviews['sentiment_analysis'] = reviews['sentiment_analysis'].map(sentiment_mapping)

# Imprimimos el DataFrame resultante
reviews.head()

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\Maverick\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


,id,app_name,user_id,funny,helpful,recommend,posted,developer,review,year,sentiment_analysis
36490,10,Counter-Strike,76561198089673598,,No ratings yet,True,"Posted December 14, 2014.",Valve,o cs e um otimo jogo de fps ele quase sempre,2014,Neutral
36463,10,Counter-Strike,Hype_R,,0 of 1 people (0%) found this review helpful,True,Posted January 30.,Valve,Best God Damn Game Known To Man Kind,2016,Positivo
36464,10,Counter-Strike,KILLERamateur,,No ratings yet,True,"Posted March 9, 2014.",Valve,I heard that when i was made it was a huge suc...,2014,Positivo
36495,10,Counter-Strike,nottynotty,,No ratings yet,True,Posted April 1.,Valve,GoODGAmP!!!,2016,Neutral
36466,10,Counter-Strike,76561198082737449,,No ratings yet,True,"Posted October 18, 2014.",Valve,realmente,2014,Neutral


In [89]:
reviews.to_parquet('DataBases/reviews_final.parquet', index=False)

In [94]:
reviews.info

<bound method DataFrame.info of            id               app_name            user_id  \
36490      10         Counter-Strike  76561198089673598   
36463      10         Counter-Strike             Hype_R   
36464      10         Counter-Strike      KILLERamateur   
36495      10         Counter-Strike         nottynotty   
36466      10         Counter-Strike  76561198082737449   
...       ...                    ...                ...   
48867  521430           Super Switch  76561198075141715   
47540  521570  You Have 10 Seconds 2  76561198071122396   
47539  521570  You Have 10 Seconds 2        TfhuAWGscvg   
48413  521990         Galactic Storm           mikeyg74   
48292  527340          What The Box?           Limo1300   

                                  funny  \
36490                                     
36463                                     
36464                                     
36495                                     
36466                                     
.